# Quotes for swaps

In [ ]:
#| default_exp quote

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

from functools import reduce
from typing import List, Union, Tuple
from eth_abi.packed import encode_packed
from dataclasses import dataclass
from sugar.token import Token
from sugar.pool import LiquidityPoolForSwap

In [ ]:
#| export 

# magic numbers
QUOTER_STABLE_POOL_FILLER, QUOTER_VOLATILE_POOL_FILLER = 2097152, 4194304

@dataclass
class PreparedRoute:
    types: List[str]; values: List[Union[str, int]]

    @property
    def encoded(self) -> bytes: return encode_packed(self.types, self.values)

def pack_path(path: List[Tuple[LiquidityPoolForSwap, bool]]) -> PreparedRoute:
    types, values = reduce(lambda s, pool: s + pool, [["address", "int24"] for i in range(len(path))], []) + ["address"], []
    for node in path:
        pool, reversed = node
        token0, token1 = pool.token0_address if not reversed else pool.token1_address, pool.token1_address if not reversed else pool.token0_address
        if pool.type > 0: filler = pool.type
        else: filler =  QUOTER_STABLE_POOL_FILLER if pool.is_stable else QUOTER_VOLATILE_POOL_FILLER
        if len(values) == 0: values = [token0, filler, token1]
        else: values += [filler, token1]
    return PreparedRoute(types=types, values=values)

@dataclass
class QuoteInput:
    from_token: Token; to_token: Token; path: List[Tuple[LiquidityPoolForSwap, bool]]; amount_in: int

    def to_tuple(self) -> tuple: return (self.from_token, self.to_token, tuple(self.path), self.amount_in, self.amount_out)

    @staticmethod
    def from_tuple(t: tuple) -> "QuoteInput":
        from_token, to_token, path, amount_in, amount_out = t
        return QuoteInput(from_token=from_token, to_token=to_token, path=list(path), amount_in=amount_in, amount_out=amount_out)
    
    @property
    def route(self) -> PreparedRoute: return pack_path(self.path)

@dataclass
class Quote:
    input: QuoteInput; amount_out: int

    @property
    def from_token(self) -> Token: return self.input.from_token
    @property
    def to_token(self) -> Token: return self.input.to_token
    @property
    def path(self) -> List[Tuple[LiquidityPoolForSwap, bool]]: return self.input.path
    @property
    def amount_in(self) -> int: return self.input.amount_in

## Tests

In [ ]:
from fastcore.test import test_eq

In [ ]:
path = [
    (LiquidityPoolForSwap(lp='0xec3d9098BD40ec741676fc04D4bd26BCCF592aa3', type=200, token0_address='0x4200000000000000000000000000000000000006', token1_address='0x6c84a8f1c29108F47a79964b5Fe888D4f4D0dE40', is_stable=False, is_cl=True), False),
    (LiquidityPoolForSwap(lp='0x02A130b6D35611bC2D90e20f2ceA45431c0A9a8d', type=-1, token0_address='0x6c84a8f1c29108F47a79964b5Fe888D4f4D0dE40', token1_address='0x7F5c764cBc14f9669B88837ca1490cCa17c31607', is_stable=False, is_cl=False), False),
    (LiquidityPoolForSwap(lp='0xf7a73e1995030B0198f8d6e528a1c42ACEf90a4c', type=0, token0_address='0x7F5c764cBc14f9669B88837ca1490cCa17c31607', token1_address='0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db', is_stable=True, is_cl=False), False)
]

rt = pack_path(path)

test_eq(rt.types, ['address', 'int24', 'address', 'int24', 'address', 'int24', 'address'])
test_eq(rt.values, ['0x4200000000000000000000000000000000000006', 200, '0x6c84a8f1c29108F47a79964b5Fe888D4f4D0dE40', 4194304, '0x7F5c764cBc14f9669B88837ca1490cCa17c31607', 2097152, '0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db'])
test_eq(rt.encoded.hex(), "42000000000000000000000000000000000000060000c86c84a8f1c29108f47a79964b5fe888d4f4d0de404000007f5c764cbc14f9669b88837ca1490cca17c316072000009560e827af36c94d2ac33a39bce1fe78631088db")

# test reversed version

path = [
    (LiquidityPoolForSwap(lp='0xec3d9098BD40ec741676fc04D4bd26BCCF592aa3', type=200, token0_address='0x4200000000000000000000000000000000000006', token1_address='0x6c84a8f1c29108F47a79964b5Fe888D4f4D0dE40', is_stable=False, is_cl=True), True),
    (LiquidityPoolForSwap(lp='0x02A130b6D35611bC2D90e20f2ceA45431c0A9a8d', type=-1, token0_address='0x6c84a8f1c29108F47a79964b5Fe888D4f4D0dE40', token1_address='0x7F5c764cBc14f9669B88837ca1490cCa17c31607', is_stable=False, is_cl=False), False),
    (LiquidityPoolForSwap(lp='0xf7a73e1995030B0198f8d6e528a1c42ACEf90a4c', type=0, token0_address='0x7F5c764cBc14f9669B88837ca1490cCa17c31607', token1_address='0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db', is_stable=True, is_cl=False), False)
]

rt = pack_path(path)

test_eq(rt.types, ['address', 'int24', 'address', 'int24', 'address', 'int24', 'address'])
test_eq(rt.values, ['0x6c84a8f1c29108F47a79964b5Fe888D4f4D0dE40', 200, '0x4200000000000000000000000000000000000006', 4194304, '0x7F5c764cBc14f9669B88837ca1490cCa17c31607', 2097152, '0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db'])
test_eq(rt.encoded.hex(), "6c84a8f1c29108f47a79964b5fe888d4f4d0de400000c842000000000000000000000000000000000000064000007f5c764cbc14f9669b88837ca1490cca17c316072000009560e827af36c94d2ac33a39bce1fe78631088db")


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()